# Setting Up Google Earth Engine API
1) __Clone your environment:__

To play it safe, you can clone your active environment.  We'll need to install some packages, and doing this incorrectly could cause issues with other projects. 
- In ArcGIS Pro, go to _Project_
- On the left hand menu, go to _Package Manager_
- In the _Package Manager_ window, select _active environments_
- In the _active environments_ window, select the clone package button
- Save your new environmrnt
- If you run into errors during the cloning process, you can try opening a blank new project, and follow the instructions again.  You may have to restart your computer after this. 

2) __Install Packages for Google Earth Engine API:__

- Go to your console terminal and open a new python window
- Ensure you are in the correct directory (the new one you just created)
- Run: conda install -c conda-forge earthengine-api

This should install the earth engine api in your new environment. 

# GEE Imagery and Mapping

1) Import packages

_Include_ os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] ="yes" _before importing geemap to avoid "Open SSL 3.0 Legacy" error_

In [1]:
import os
import arcpy
# This is to resolve an error that occured during import "SSL 3.0 Legacy"
os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] ="yes" 
import os.path
import google
import geemap #Custom library to map earth engine data
import ee
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
import pprint
import branca.colormap as cm
import datetime
import palettable as cp
import traceback
from IPython.display import HTML
from IPython.display import Image
from arcgis.gis import GIS
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from palettable.matplotlib import Viridis_4


We're using Google Earth Engine to access sattelite imagery from the Sentinel-2 sattellites, so we'll need to establish a connection by authorizing client access to Google Earth Engine.  Initially, this prompts a google acount login, which is saved as a json token for authenticating subsequent session.

ee.Authenticate prompts authorization access to Earth Engine via OAuth2.  You can read more about OAuth2 to access google APIs <a href="https://developers.google.com/identity/protocols/oauth2">here</a>

In [2]:
ee.Authenticate()

True

Connect to an existing Google Cloud Project, this requires you to set one up, which you can read more about <a href="https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup">here</a>

In [3]:
ee.Initialize(project = 'byargeon-algae-2024')

Specify all input variables

In [4]:
START_DATE = ee.Date('2022-01-01') # Initial date for our search range
END_DATE = ee.Date('2024-01-01') #Final Date for our search range
CLOUD_FILTER = 60 
CLD_PRB_THRESH = 50 #Integer cloud probability 

# Define AOI Polygon that includes the highland lake region of Central Texas
AOI = ee.Geometry.Polygon(
        [[[-98.58221017603384, 30.94588417271635],
          [-98.58221017603384, 30.134509757818982],
          [-97.56597482447134, 30.134509757818982],
          [-97.56597482447134, 30.94588417271635]]])

Collect Sentinel-2 imagery, filter by the AOI, the provided date range, and images with > 60% cloud cover.  This produces a collection of images, so take the median of the collection, resulting in a single image. 

The Cloud Filter is using the "Cloudy Pixel Percentage" band to filter out cloudy pixels

In [5]:
S2 = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(AOI).filterDate(START_DATE, END_DATE).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).median())

C:\Users\travisbyargeon\AppData\Local\ESRI\conda\envs\arcgispro-py3-ee\lib\site-packages\ee\deprecation.py:202: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Create a map to display our indices, center the map near Lake Travis in Travis County, TX.  Set the zoom level (higher = more zoom)

In [6]:
Map = geemap.Map(center=[30.3825, -97.915], zoom=11)

Calculate the spectral indices for the collected S2 Image.  We do this by using a the ee funtion image.normalizedDifference which calculated the normalized difference between two bands.  This calculation renders the following imagery:

Normalized Difference Water Index (NDWI) is used to visualize water
= (B3 - B11)/(B3 + B11)

Normalized Difference Chlorophyll Index (NDWI) is used to visualize chlorophyll
= (B5 - B4)/(B5 + B4)

Normalized Difference Vegetation Index (NDWI) is used to visualize vegetation
= (B8 - B4)/(B8 + B4)

In [7]:
ndwi = S2.normalizedDifference(['B3','B11'])
ndci = S2.normalizedDifference(['B5','B4'])
ndvi = S2.normalizedDifference(['B8','B4'])

We can use the NDWI to create a water/land mask for cartographic purposes. I determined the value of 0.01 by trial and error. 

In [8]:
land_mask = ndwi.gte(0.01)
water_mask = ndwi.lte(0.01)


Apply the masks to our spectral incides for cartographic purposes

In [9]:
ndwi_masked = ndwi.updateMask(land_mask)
ndci_masked = ndci.updateMask(land_mask)
ndvi_masked = ndvi.updateMask(water_mask).updateMask(ndvi.gte(0.09))

Specify the visualization paramters for our images

In [10]:
rbg_viz = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': [0.95,1.1,1]}
nir_viz = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 2500, 'gamma': [0.95,1.1,1]}
bat_viz = {'bands': ['B4', 'B3', 'B1'], 'min': 0, 'max': 2500, 'gamma': [0.95,1.1,1]}
ndwi_viz = {'min': 0.0, 'max': 0.05, 'palette':['00FFFF','0000FF']}
ndci_viz = {'min': -0.003, 'max': 0.03, 'palette':['blue','green','yellow']}
ndvi_viz = {'min': -0.5, 'max': 0.5, 'palette':['blue','white','green']}

Add all layers to our map

In [11]:
Map.add_layer(S2, rbg_viz, 'Natural')
Map.add_layer(S2, nir_viz, 'Infrared')
#Map.add_layer(S2, bat_viz, 'Bathymetric')
Map.add_layer(ndwi_masked, ndwi_viz, 'NDWI')
Map.add_layer(ndci_masked, ndci_viz, 'NDCI')
Map.add_layer(ndvi_masked, ndvi_viz, 'NDVI')

Display the Map

In [12]:
Map

Map(center=[30.3825, -97.915], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…